# 线性代数

In [6]:
import torch
from torch import nn
from torch.utils import data

## 生成数据

In [7]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

true_w = torch.tensor([-2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features' ,features[0],'\nlabels' ,labels[0])

def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

features tensor([ 0.0426, -1.0705]) 
labels tensor([7.7576])


## 创建神经网络

In [9]:
# 定义一个序列神经网络
net = nn.Sequential(nn.Linear(2, 1))
print('net:',net)

# 初始化第一层线性函数的数据
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

# 创建一个标准差损失函数
loss = nn.MSELoss()

# 定义优化算法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

# 检查结果
w = net[0].weight.data
print('w:',w.reshape(true_w.shape))
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b:',b)
print('b的估计误差：', true_b - b)

net: Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
epoch 1, loss 0.000400
epoch 2, loss 0.000099
epoch 3, loss 0.000101
w: tensor([-1.9983, -3.4001])
w的估计误差： tensor([-0.0017,  0.0001])
b: tensor([4.2005])
b的估计误差： tensor([-0.0005])
